# Chart Pattern Detection

This notebook demonstrates numta's chart pattern detection capabilities, including head and shoulders, double/triple patterns, triangles, wedges, flags, and VCP patterns.

In [ ]:
import numpy as np
import pandas as pd
import numta
from numta import (
    # Swing detection
    find_swing_highs, find_swing_lows, find_swing_points,
    # Pattern detection
    detect_head_shoulders, detect_inverse_head_shoulders,
    detect_double_top, detect_double_bottom,
    detect_triple_top, detect_triple_bottom,
    detect_triangle, detect_wedge, detect_flag, detect_vcp
)

## Creating Sample Data

We'll generate price data with some embedded patterns.

In [ ]:
np.random.seed(42)
n = 300

# Generate a price series with trends
trend = np.sin(np.linspace(0, 4*np.pi, n)) * 20
noise = np.cumsum(np.random.randn(n) * 0.5)
close = 100 + trend + noise

# Create OHLC data
df = pd.DataFrame({
    'open': close + np.random.randn(n) * 0.5,
    'high': close + np.abs(np.random.randn(n)) * 1.5,
    'low': close - np.abs(np.random.randn(n)) * 1.5,
    'close': close,
    'volume': np.random.randint(1000, 10000, n).astype(float)
})

print(f"Sample data: {n} bars")
print(f"Price range: {df['close'].min():.2f} - {df['close'].max():.2f}")

## Swing Point Detection

Before detecting patterns, we need to identify swing highs and lows.

In [ ]:
# Find swing highs and lows
highs = df['high'].values
lows = df['low'].values

# order parameter controls the number of bars on each side
swing_highs = find_swing_highs(highs, order=5)
swing_lows = find_swing_lows(lows, order=5)

print(f"Swing highs found: {len(swing_highs)}")
print(f"Swing lows found: {len(swing_lows)}")

# Get combined swing points
swing_points = find_swing_points(highs, lows, order=5)
print(f"\nTotal swing points: {len(swing_points)}")

## Head and Shoulders

The head and shoulders pattern is a reversal pattern consisting of three peaks, with the middle peak (head) being the highest.

### Regular Head and Shoulders (Bearish)

In [ ]:
# Detect head and shoulders patterns
hs_patterns = detect_head_shoulders(highs, lows, order=5)

print(f"Head and Shoulders patterns found: {len(hs_patterns)}")

for pattern in hs_patterns[:3]:  # Show first 3
    print(f"\nPattern at indices {pattern.start_index} - {pattern.end_index}")
    print(f"  Confidence: {pattern.confidence:.2f}")
    print(f"  Left shoulder: {pattern.left_shoulder_index}")
    print(f"  Head: {pattern.head_index}")
    print(f"  Right shoulder: {pattern.right_shoulder_index}")
    print(f"  Neckline: {pattern.neckline_slope:.4f}")

### Inverse Head and Shoulders (Bullish)

In [ ]:
# Detect inverse head and shoulders patterns
ihs_patterns = detect_inverse_head_shoulders(highs, lows, order=5)

print(f"Inverse Head and Shoulders patterns found: {len(ihs_patterns)}")

for pattern in ihs_patterns[:3]:
    print(f"\nPattern at indices {pattern.start_index} - {pattern.end_index}")
    print(f"  Confidence: {pattern.confidence:.2f}")

## Double and Triple Patterns

### Double Top and Double Bottom

In [ ]:
# Double Top (bearish reversal)
double_tops = detect_double_top(highs, lows, order=5)
print(f"Double Top patterns: {len(double_tops)}")

for pattern in double_tops[:2]:
    print(f"  Indices: {pattern.start_index} - {pattern.end_index}, Confidence: {pattern.confidence:.2f}")

In [ ]:
# Double Bottom (bullish reversal)
double_bottoms = detect_double_bottom(highs, lows, order=5)
print(f"Double Bottom patterns: {len(double_bottoms)}")

for pattern in double_bottoms[:2]:
    print(f"  Indices: {pattern.start_index} - {pattern.end_index}, Confidence: {pattern.confidence:.2f}")

### Triple Top and Triple Bottom

In [ ]:
# Triple Top (bearish reversal)
triple_tops = detect_triple_top(highs, lows, order=5)
print(f"Triple Top patterns: {len(triple_tops)}")

# Triple Bottom (bullish reversal)
triple_bottoms = detect_triple_bottom(highs, lows, order=5)
print(f"Triple Bottom patterns: {len(triple_bottoms)}")

## Triangles

Triangles are continuation patterns characterized by converging trendlines.

In [ ]:
# Detect triangles (ascending, descending, symmetrical)
triangles = detect_triangle(highs, lows, order=5)

print(f"Triangle patterns found: {len(triangles)}")

for pattern in triangles[:3]:
    print(f"\n{pattern.triangle_type.title()} Triangle")
    print(f"  Indices: {pattern.start_index} - {pattern.end_index}")
    print(f"  Confidence: {pattern.confidence:.2f}")
    print(f"  Upper trendline slope: {pattern.upper_trendline_slope:.4f}")
    print(f"  Lower trendline slope: {pattern.lower_trendline_slope:.4f}")

## Wedges

Wedges are similar to triangles but both trendlines slope in the same direction.

In [ ]:
# Detect wedges (rising, falling)
wedges = detect_wedge(highs, lows, order=5)

print(f"Wedge patterns found: {len(wedges)}")

for pattern in wedges[:3]:
    print(f"\n{pattern.wedge_type.title()} Wedge")
    print(f"  Indices: {pattern.start_index} - {pattern.end_index}")
    print(f"  Confidence: {pattern.confidence:.2f}")

## Flags and Pennants

Flags are continuation patterns that form after a strong move.

In [ ]:
# Detect flags and pennants
flags = detect_flag(highs, lows, close, order=5)

print(f"Flag patterns found: {len(flags)}")

for pattern in flags[:3]:
    print(f"\n{pattern.flag_type.title()}")
    print(f"  Indices: {pattern.start_index} - {pattern.end_index}")
    print(f"  Pole direction: {pattern.pole_direction}")
    print(f"  Confidence: {pattern.confidence:.2f}")

## Volatility Contraction Pattern (VCP)

The VCP is characterized by decreasing price swings (volatility contraction).

In [ ]:
# Detect VCP patterns
vcp_patterns = detect_vcp(highs, lows, order=5)

print(f"VCP patterns found: {len(vcp_patterns)}")

for pattern in vcp_patterns[:3]:
    print(f"\nVCP Pattern")
    print(f"  Indices: {pattern.start_index} - {pattern.end_index}")
    print(f"  Contraction count: {pattern.contraction_count}")
    print(f"  Confidence: {pattern.confidence:.2f}")

## Pattern Confidence Scores

All pattern detection functions return a confidence score between 0 and 1. Higher scores indicate patterns that more closely match the ideal characteristics.

In [ ]:
# Combine all patterns and sort by confidence
all_patterns = []

for p in hs_patterns:
    all_patterns.append(('Head & Shoulders', p.confidence, p.start_index, p.end_index))

for p in ihs_patterns:
    all_patterns.append(('Inverse H&S', p.confidence, p.start_index, p.end_index))

for p in double_tops:
    all_patterns.append(('Double Top', p.confidence, p.start_index, p.end_index))

for p in double_bottoms:
    all_patterns.append(('Double Bottom', p.confidence, p.start_index, p.end_index))

for p in triangles:
    all_patterns.append((f'{p.triangle_type} Triangle', p.confidence, p.start_index, p.end_index))

# Sort by confidence
all_patterns.sort(key=lambda x: x[1], reverse=True)

print("Top patterns by confidence:")
for name, conf, start, end in all_patterns[:10]:
    print(f"  {name}: {conf:.3f} (bars {start}-{end})")

## Using the Pandas Accessor

Pattern detection is also available through the `.ta` accessor.

In [ ]:
# Find all patterns
patterns = df.ta.find_patterns(pattern_type='all', order=5)
print(f"Total patterns found: {len(patterns)}")

# Find specific pattern types
double_patterns = df.ta.find_patterns(pattern_type='double')
print(f"Double patterns: {len(double_patterns)}")

triangles = df.ta.find_patterns(pattern_type='triangle')
print(f"Triangle patterns: {len(triangles)}")

head_shoulders = df.ta.find_patterns(pattern_type='head_shoulders')
print(f"Head & Shoulders patterns: {len(head_shoulders)}")

## Next Steps

- See `05_harmonic_patterns.ipynb` for harmonic pattern recognition (Gartley, Butterfly, etc.)
- See `07_visualization.ipynb` for visualizing patterns with lwcharts